In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model
from Source import golay

Using TensorFlow backend.


In [2]:
G = golay.make_G()

In [3]:
def awgn(x_input, reqSNR):
    sig_power = 1
    
    noise_power = np.sqrt(sig_power / (2*reqSNR))
    noise = noise_power * np.random.normal(size = x_input.shape)
    
    return x_input + noise

In [4]:
model = load_model('model/add_2_(24,12)_p6.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [5]:
for i in range(1000):
    train_size = 10**6
    m = np.random.randint(2, size = (train_size, 12))
    train = np.dot(m,G) % 2

    odd = np.where(train.sum(axis=1) % 2 == 1)
    even_check = np.zeros((train_size, 1), dtype='int')
    even_check[odd] = 1
    train = np.concatenate((even_check, train), axis=1)
    train = np.where(train > 0 , -1, 1)

    Eb_No = 10**(1/10)
    Es_No = Eb_No * (12/24)
    train_noise = awgn(train[:25*10**4], Es_No)

    Eb_No = 10**(4/10)
    Es_No = Eb_No * (12/24)
    train_noise = np.vstack((train_noise, awgn(train[25*10**4:50*10**4], Es_No)))

    Eb_No = 10**(7/10)
    Es_No = Eb_No * (12/24)
    train_noise = np.vstack((train_noise, awgn(train[50*10**4:75*10**4], Es_No)))

    Eb_No = 10**(9/10)
    Es_No = Eb_No * (12/24)
    train_noise = np.vstack((train_noise, awgn(train[75*10**4:], Es_No)))

    masking = np.ones((train_size, 2))
    masking[25*10**4:50*10**4, 1] = -1*masking[25*10**4:50*10**4, 1]
    masking[50*10**4:75*10**4, 0] = -1*masking[50*10**4:75*10**4, 0] 
    masking[75*10**4:] = -1*masking[75*10**4:]
    train = np.hstack((masking, train))
    train_noise = np.hstack((masking, train_noise))
    
    
    test_size = 10**3
    m = np.random.randint(2, size = (test_size, 12))
    test = np.dot(m,G) % 2

    odd = np.where(test.sum(axis=1) % 2 == 1)
    even_check = np.zeros((test_size, 1), dtype='int')
    even_check[odd] = 1
    test = np.concatenate((even_check, test), axis=1)
    test = np.where(test > 0 , -1, 1)

    Eb_No = 10**(1/10)
    Es_No = Eb_No * (12/24)
    test_noise = awgn(test[:25*10], Es_No)

    Eb_No = 10**(4/10)
    Es_No = Eb_No * (12/24)
    test_noise = np.vstack((test_noise, awgn(test[25*10:50*10], Es_No)))

    Eb_No = 10**(7/10)
    Es_No = Eb_No * (12/24)
    test_noise = np.vstack((test_noise, awgn(test[50*10:75*10], Es_No)))

    Eb_No = 10**(9/10)
    Es_No = Eb_No * (12/24)
    test_noise = np.vstack((test_noise, awgn(test[75*10:], Es_No)))

    masking = np.ones((test_size, 2))
    masking[25*10:50*10, 1] = -1*masking[25*10:50*10, 1]
    masking[50*10:75*10, 0] = -1*masking[50*10:75*10, 0] 
    masking[75*10:] = -1*masking[75*10:]
    test = np.hstack((masking, test))
    test_noise = np.hstack((masking, test_noise))
    
    model.fit(train_noise, train,
          epochs=100,
          batch_size=10000,
          shuffle=True,
          validation_data=(test_noise, test),verbose=0)    
    
    if i%50 == 0:
        loss = model.evaluate(test_noise, test)
        print(loss)
        
        model.save('model/add_2_(24,12)_p6.h5') 

1000/1000 [==============================] - 0s 93us/step
[0.07438104187743738, 0.09000000357627869]
1000/1000 [==============================] - 0s 105us/step
[0.07897881784051423, 0.0949999988079071]
1000/1000 [==============================] - 0s 103us/step
[0.07619704168115277, 0.13500000536441803]
1000/1000 [==============================] - 0s 89us/step
[0.07887312905152793, 0.1080000028014183]
1000/1000 [==============================] - 0s 105us/step
[0.08395266327331774, 0.11999999731779099]
1000/1000 [==============================] - 0s 104us/step
[0.09678210863471032, 0.20999999344348907]
1000/1000 [==============================] - 0s 119us/step
[0.08578365169791505, 0.13699999451637268]
1000/1000 [==============================] - 0s 113us/step
[0.08698067876254209, 0.08100000023841858]
1000/1000 [==============================] - 0s 98us/step
[0.08877623003441841, 0.04800000041723251]
1000/1000 [==============================] - 0s 114us/step
[0.08774176995502785, 0.1180

KeyboardInterrupt: 

In [ ]:
model.save('model/add_2_(24,12)_p6.h5')